In [3]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# Root directory of the project
ROOT_DIR = 'D:/research/DiabeticPN/ResNet50/Mask_RCNN-master'
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from samples.DiabeticPN import custom

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

custom_WEIGHTS_PATH = "D:/research/DiabeticPN/ResNet50/Mask_RCNN-master/logs/wound20181217T1228/mask_rcnn_wound_0015.h5"  # TODO: update this path

In [4]:
# Dataset directory
custom_DIR = 'D:/research/DiabeticPN/DataSet/MaskRCNNData'
# Inference Configuration
config = custom.CustomConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [5]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# Only inference mode is supported right now
TEST_MODE = "inference"

In [6]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    fig.tight_layout()
    return ax

In [7]:
# Load validation dataset
dataset = custom.CustomDataSet()
dataset.load_custom(custom_DIR, "val")

# # Must call before using the dataset
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 61
Classes: ['BG', 'wound']


In [8]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [9]:
# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)# load the last model you trained
# weights_path = model.find_last()[1]


Loading weights  D:/research/DiabeticPN/ResNet50/Mask_RCNN-master/logs/wound20181217T1228/mask_rcnn_wound_0015.h5
Re-starting from epoch 15


In [ ]:
image_id = random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))
print("Original image shape: ", modellib.parse_image_meta(image_meta[np.newaxis,...])["original_image_shape"][0])

# Run object detection
results = model.detect([image], verbose=1)
#results = model.detect_molded(np.expand_dims(image, 0), np.expand_dims(image_meta, 0), verbose=1)

# Display results
r = results[0]
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

# Compute AP over range 0.5 to 0.95 and print it
utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
                       r['rois'], r['class_ids'], r['scores'], r['masks'],
                       verbose=1)

visualize.display_differences(
    image,
    gt_bbox, gt_class_id, gt_mask,
    r['rois'], r['class_ids'], r['scores'], r['masks'],
    dataset.class_names, ax=get_ax(),
    show_box=False, show_mask=False,
    iou_threshold=0.5, score_threshold=0.5)

image ID: wound.Grade0_49_resized.jpg (2) D:/research/DiabeticPN/DataSet/MaskRCNNData\val\Grade0_49_resized.jpg
Original image shape:  [300 300   3]
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


In [14]:
# Display predictions only
# visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset.class_names, r['scores'], ax=get_ax(1),
#                             show_bbox=False, show_mask=False,
#                             title="Predictions")

In [15]:
# Display Ground Truth only
# visualize.display_instances(image, gt_bbox, gt_mask, gt_class_id, 
#                             dataset.class_names, ax=get_ax(1),
#                             show_bbox=False, show_mask=False,
#                             title="Ground Truth")

In [13]:
def compute_batch_ap(dataset, image_ids, verbose=1):
    APs = []
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect_molded(image[np.newaxis], image_meta[np.newaxis], verbose=0)
        # Compute AP over range 0.5 to 0.95
        r = results[0]
        ap = utils.compute_ap_range(
            gt_bbox, gt_class_id, gt_mask,
            r['rois'], r['class_ids'], r['scores'], r['masks'],
            verbose=0)
        APs.append(ap)
        if verbose:
            info = dataset.image_info[image_id]
            meta = modellib.parse_image_meta(image_meta[np.newaxis,...])
            print("{:3} {}   AP: {:.2f}".format(
                meta["image_id"][0], meta["original_image_shape"][0], ap))
    return APs

# Run on validation set
limit = 5
APs = compute_batch_ap(dataset, dataset.image_ids)

  0 [300 300   3]   AP: 0.00
  1 [300 300   3]   AP: 0.90
  2 [300 300   3]   AP: 0.50
  3 [300 300   3]   AP: 0.80
  4 [300 300   3]   AP: 0.80
  5 [300 300   3]   AP: 0.20
  6 [300 300   3]   AP: 0.30
  7 [300 300   3]   AP: 0.50
  8 [300 300   3]   AP: 0.50
  9 [300 300   3]   AP: 0.43
 10 [300 300   3]   AP: 0.60
 11 [300 300   3]   AP: 0.80
 12 [300 300   3]   AP: 0.15
 13 [300 300   3]   AP: 0.00
 14 [300 300   3]   AP: 0.00
 15 [300 300   3]   AP: 0.00
 16 [300 300   3]   AP: 0.00
 17 [300 300   3]   AP: 0.05
 18 [300 300   3]   AP: 0.00
 19 [300 300   3]   AP: 0.06
 20 [300 300   3]   AP: 0.00
 21 [300 300   3]   AP: 0.20
 22 [300 300   3]   AP: 0.00
 23 [300 300   3]   AP: 0.05
 24 [300 300   3]   AP: 0.20
 25 [300 300   3]   AP: 0.00
 26 [300 300   3]   AP: 0.40
 27 [300 300   3]   AP: 0.00
 28 [300 300   3]   AP: 0.00
 29 [300 300   3]   AP: 0.40
 30 [300 300   3]   AP: 0.00
 31 [300 300   3]   AP: 0.40
 32 [300 300   3]   AP: 0.00
 33 [300 300   3]   AP: 0.10
 34 [300 300  

In [14]:
print("Mean AP overa {} images: {:.4f}".format(len(APs), np.mean(APs)))

Mean AP overa 61 images: 0.2909
